In [1]:
import os
import sys
import json

sys.path.append('../')

from src.pipelines.extraction import extract_information_from_cv, extract_information_from_cv_gemini,extract_information_from_job_description

c:\Users\yassi\Desktop\SmartRecru\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
dommy_cv='''Jean Dupont
Ingénieur Logiciel

Contact:
Email: jean.dupont@example.com
Téléphone: 06 12 34 56 78

Résumé:
Ingénieur logiciel expérimenté, passionné par le développement de programmes innovants qui accélèrent l'efficacité et le succès de l'organisation.

Compétences:
- Python
- Java
- C++
- JavaScript
- SQL
- Git

Expérience:
Ingénieur Logiciel chez Google (2020-Présent)
- Développement et maintenance d'applications web en utilisant Python et Django.
- Collaboration avec des équipes inter-fonctionnelles pour définir, concevoir et livrer de nouvelles fonctionnalités.'''

dummy_job_offer='''Titre: Développeur Python Senior

Entreprise: Tech Solutions Inc.

Lieu: Paris, France

Description:
Nous recherchons un développeur Python senior pour rejoindre notre équipe. Le candidat idéal aura une solide expérience dans le développement d'applications web et une connaissance approfondie de l'écosystème Python.

Responsabilités:
- Concevoir, développer et maintenir des applications web robustes.
- Écrire du code propre, maintenable et efficace.
- Participer aux revues de code et au mentorat des développeurs juniors.

Qualifications:
- 5+ ans d'expérience en développement Python.
- Expérience avec des frameworks web comme Django ou Flask.
- Solide connaissance de SQL et des bases de données relationnelles.
- Maîtrise de Git.'''

In [3]:
print("--- CV Extraction (spaCy/Regex) ---")
extracted_info_en = extract_information_from_cv(dommy_cv)
print(json.dumps(extracted_info_en, indent=2))

--- CV Extraction (spaCy/Regex) ---
{
  "name": "Jean Dupont",
  "contact": {
    "emails": [
      "jean.dupont@example.com"
    ],
    "phones": []
  },
  "skills": "",
  "education": "",
  "experience": "",
  "academic_projects": "",
  "diplomas": ""
}


In [4]:
print("\n--- Job Offer Extraction (spaCy/Regex) ---")
extracted_job_info = extract_information_from_job_description(dummy_job_offer)
print(json.dumps(extracted_job_info, indent=2))


--- Job Offer Extraction (spaCy/Regex) ---
{
  "skills_and_requirements": ""
}


In [5]:
# --- Gemini Extraction Example ---
print("\n--- CV Extraction (Gemini) ---")
gemini_info = extract_information_from_cv_gemini(dommy_cv)
if gemini_info:
        print(json.dumps(gemini_info, indent=2))



--- CV Extraction (Gemini) ---
{
  "name": "Jean Dupont",
  "contact": {
    "email": "jean.dupont@example.com",
    "phone": "06 12 34 56 78"
  },
  "education": [],
  "experience": [
    "Ing\u00e9nieur Logiciel chez Google (2020-Pr\u00e9sent) - D\u00e9veloppement et maintenance d'applications web en utilisant Python et Django. - Collaboration avec des \u00e9quipes inter-fonctionnelles pour d\u00e9finir, concevoir et livrer de nouvelles fonctionnalit\u00e9s."
  ],
  "skills": [
    "Python",
    "Java",
    "C++",
    "JavaScript",
    "SQL",
    "Git"
  ],
  "academic_projects": [],
  "diplomas": []
}


In [6]:
# --- Gemini Extraction Example ---
from src.pipelines.extraction import extract_information_from_jd_gemini


print("\n--- CV Extraction (Gemini) ---")
jb_info = extract_information_from_jd_gemini(dummy_job_offer)
if jb_info:
        print(json.dumps(jb_info, indent=2))


--- CV Extraction (Gemini) ---
{
  "job_title": "D\u00e9veloppeur Python Senior",
  "company_name": "Tech Solutions Inc.",
  "location": "Paris, France",
  "job_type": null,
  "responsibilities": [
    "Concevoir, d\u00e9velopper et maintenir des applications web robustes.",
    "\u00c9crire du code propre, maintenable et efficace.",
    "Participer aux revues de code et au mentorat des d\u00e9veloppeurs juniors."
  ],
  "skills": [
    "d\u00e9veloppement Python",
    "frameworks web comme Django ou Flask",
    "SQL",
    "bases de donn\u00e9es relationnelles",
    "Git"
  ],
  "experience_level": "5+ ans d'exp\u00e9rience en d\u00e9veloppement Python",
  "education_requirements": []
}


In [7]:
from src.utils.embedding_generator import generate_embeddings

print("--- Embedding Generation ---")
cv_embeddings = generate_embeddings(dommy_cv)
job_offer_embeddings = generate_embeddings(dummy_job_offer)
print(f"CV Embeddings Shape: {cv_embeddings.shape}")
print(f"Job Offer Embeddings Shape: {job_offer_embeddings.shape}")

Using device: cpu
--- Embedding Generation ---
CV Embeddings Shape: (1, 384)
Job Offer Embeddings Shape: (1, 384)


In [8]:
from src.core.matching import calculate_fit_score

print("--- Fit Score Calculation ---")
fit_score = calculate_fit_score(cv_embeddings, job_offer_embeddings)
print(f"Fit Score: {fit_score}")

--- Fit Score Calculation ---
Fit Score: 0.6608319282531738


In [9]:
from src.core.matching import skill_gap_analysis       
print("-- Skill Gap Analysis ---")  
cv_skills= gemini_info["skills"]
job_offer_skills= jb_info["skills"]
missing_skills, matched_skills  = skill_gap_analysis(cv_skills, job_offer_skills)
print(cv_skills)
print(job_offer_skills)
print(missing_skills)
print(matched_skills)

-- Skill Gap Analysis ---
['Python', 'Java', 'C++', 'JavaScript', 'SQL', 'Git']
['développement Python', 'frameworks web comme Django ou Flask', 'SQL', 'bases de données relationnelles', 'Git']
['frameworks web comme Django ou Flask', 'bases de données relationnelles']
['développement Python', 'SQL', 'Git']


In [10]:
from src.core.summarization import summarize_cv, summarize_jd, generate_strengths_and_weaknesses_summary
print("--- CV Summary ---")
cv_summary = summarize_cv(dommy_cv)
print(cv_summary)
print("--- Job Offer Summary ---")
job_offer_summary = summarize_jd(dummy_job_offer)
print(job_offer_summary)
print("--- Strengths and Weaknesses Summary ---")
summary=generate_strengths_and_weaknesses_summary(dommy_cv,dummy_job_offer,matched_skills,missing_skills)
print(summary)

--- CV Summary ---
Jean Dupont est un ingénieur logiciel expérimenté, actuellement chez Google depuis 2020. Il y est chargé du développement et de la maintenance d'applications web, principalement avec Python et Django, et collabore étroitement avec des équipes inter-fonctionnelles pour la conception et la livraison de nouvelles fonctionnalités. Ses compétences techniques clés incluent Python, Java, C++, JavaScript, SQL et Git, démontrant sa capacité à créer des programmes innovants pour l'efficacité organisationnelle.
--- Job Offer Summary ---
Tech Solutions Inc. recherche un Développeur Python Senior pour concevoir, développer et maintenir des applications web robustes, en rédigeant du code propre et en participant aux revues de code et au mentorat. Les qualifications requises incluent au moins 5 ans d'expérience en développement Python, une expertise des frameworks web (Django/Flask), une solide connaissance de SQL et des bases de données relationnelles, ainsi que la maîtrise de Git

In [ ]:
from src.utils.rag_pipeline import RAGPipeline

print("--- RAG Indexing ---")
rag_pipeline = RAGPipeline()
documents_to_index = [dommy_cv]
metadatas = [{"source": "dummy_cv.txt"}]
rag_pipeline.index_documents(documents_to_index, metadatas)

# Verify the number of items in the collection
collection_count = rag_pipeline.collection.count()
print(f"Number of items in the collection: {collection_count}")

In [ ]:
from src.core.qa import answer_question

print("\n--- RAG Q&A ---")
question = "What is the candidate's experience with Python?"
answer = answer_question(question, rag_pipeline)
print(f"Question: {question}")
print(f"Answer: {answer}")